## Notebook for use case digital posting assistant - stage1
### Module A vectorize accounting assignment guide
#### Ojectives
- In this module we will develop the load and the vectorization of the text-file for the accounting assignment guide
- The vectorized accounting assignemnt guide will finaly stored in a SAP HANA vector database

#### Processing steps from concept
A0 - preparation

A2 - load and splitt: load the pdf-file containing the accounting assignment guide data from a folder and splitt the data into text_chunks

A3 - vectorize and embedd: vectorize the splitted data with embedding function. Use an embedding function to convert the text chunks into vector representations

A4 - store: create/clear a sap hana database - table and store the vector in this table

### A0 - Setup and configuration Modul A

The following setup-steps where processed:

* A0.0 Start SAP instances
* A0.1 install py-packages
* A0.2 load env-variables from config.json-file
* A0.3 Setup and test connection to HANA DB
* A0.4 Setup LLM-Connection to SAP AI-HUB


### A0.0 Start SAP Instances

* Start BTP Cockpit
* Start SAP Build Dev Space
* Start HANA DB

In [ ]:
# A0.1 install py-packages
# RESET KERNEL AFTER INSTALLATION

%pip install --upgrade pip

%pip install hdbcli --break-system-packages
%pip install generative-ai-hub-sdk[all] --break-system-packages
%pip install folium --break-system-packages
%pip install ipywidgets --break-system-packages
%pip install pypdf
%pip install -U ipykernel
%pip install hana-ml
%pip install langchain
%pip install hdbcli
%pip install sqlalchemy-hana

In [4]:
# A0.2 load env-variables from config.json-file
# This script loads environment variables from a JSON configuration file
# and sets them in the current environment. It raises an error if the file does not exist
# or if the JSON file is malformed.

import json
import os


def load_env_variables(config_file):
    """
    Load environment variables from a JSON configuration file.

    Args:
        config_file (str): Path to the JSON configuration file.

    Returns:
        dict: A dictionary containing the environment variables.
    """
    if not os.path.exists(config_file):
        raise FileNotFoundError(f"The configuration file {config_file} does not exist.")
    
    try:
        with open(config_file, 'r') as file:
            env_variables = json.load(file)
    except json.JSONDecodeError as e:
        raise ValueError(f"Error decoding JSON from the configuration file {config_file}: {e}")
    
    for key, value in env_variables.items():
        # Convert non-string values to strings before setting them in os.environ
        if isinstance(value, dict):
            value = json.dumps(value)  # Convert dictionaries to JSON strings
        os.environ[key] = str(value)
    
    return env_variables

# Example usage
config_file = "/home/user/.aicore/config.json"
try:
    env_variables = load_env_variables(config_file)
    print(f"Loaded environment variables: {env_variables}")
except (FileNotFoundError, ValueError) as e:
    print(e)

Loaded environment variables: {'AICORE_AUTH_URL': 'https://sap-ai.authentication.eu10.hana.ondemand.com', 'AICORE_CLIENT_ID': 'sb-6ebfacf0-d811-447e-9262-b18974ccd67e!b364403|aicore!b540', 'AICORE_CLIENT_SECRET': '17e8de41-d715-4cfe-b086-9c59302640df$_9OJ4fWV28VeLV5uSMyxA_DfSXHna3dEE9-0KocfNeE=', 'AICORE_RESOURCE_GROUP': 'default', 'AICORE_BASE_URL': 'https://api.ai.prod.eu-central-1.aws.ml.hana.ondemand.com/v2', 'AICORE_DEPLOYMENT_ID': 'dc4204c43f309ca1', 'AICORE_DEPLOYMENT_URL': 'https://api.ai.prod.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/dc4204c43f309ca1', 'AICORE_DEPLOYMENT_MODEL': 'gpt-4', 'AICORE_DEPLOYMENT_MODEL_VERSION': 'latest', 'AICORE_DEPLOYMENT_MODEL_EMBEDDING': 'text-embedding-ada-002', 'hdb_host_address': 'bfff8255-c34a-41cb-a822-bf9b5f56fb16.hna0.prod-eu20.hanacloud.ondemand.com', 'hdb_user': 'DBADMIN', 'hdb_password': 'DB@hmx04', 'hdb_port': '443', 'hdb_table_name': 'ACCOUNTING_ASSIGN_SUPPORT_TABLE'}


In [5]:
# A0.2 Test connection with env-Variables to SAP AI core

from gen_ai_hub.proxy.native.openai import embeddings

response = embeddings.create(
    input="SAP Generative AI Hub is awesome!",
    model_name="text-embedding-ada-002"
    
)
print(response.data)

[Embedding(embedding=[-0.001376426313072443, -0.022090168669819832, -0.005044370889663696, -0.016446400433778763, -0.03345852345228195, 0.014897393994033337, -0.008075034245848656, -0.014250853098928928, 0.0055023375898599625, -0.05840425193309784, 0.01485698577016592, -0.0007850260008126497, 0.01038507279008627, -0.02420489862561226, -0.0006524344789795578, 0.005734688602387905, 0.027828224003314972, -0.004690793342888355, 0.009731796570122242, -0.012425719760358334, -0.021106885746121407, 0.027464544400572777, 0.007381348870694637, -0.019557880237698555, -0.02378734014928341, 0.022548135370016098, 0.020366057753562927, -0.022898346185684204, -0.00685603404417634, -0.020891372114419937, 0.0011895353673025966, 0.006590009201318026, -0.021942002698779106, -0.005320497788488865, -0.001679492648690939, 0.01465494092553854, -0.0078123765997588634, -0.005374376196414232, -0.005724586546421051, 0.0041621108539402485, 0.019625229761004448, 0.0029229065403342247, -0.0027882102876901627, -0.028

In [6]:
# A0.3 Setup and test connection to HANA DB

import os
# from hana_ml import ConnectionContext
from hdbcli import dbapi

# Fetch environment variables
hdb_host_address = os.getenv("hdb_host_address")
hdb_user = os.getenv("hdb_user")
hdb_password = os.getenv("hdb_password")
hdb_port = os.getenv("hdb_port")

# Debugging: Print non-sensitive environment variables
print(f"hdb_host_address: {hdb_host_address}")
print(f"hdb_user: {hdb_user}")
print(f"hdb_port: {hdb_port}")

# Ensure variables are defined
if not all([hdb_host_address, hdb_user, hdb_password, hdb_port]):
    raise ValueError("One or more HANA DB connection parameters are missing.")

# Convert port to integer
hdb_port = int(hdb_port)

# Create a connection to the HANA database
# hana_connection = ConnectionContext(
#     address=hdb_host_address,
#     port=hdb_port,
#     user=hdb_user,
#     password=hdb_password,
#     encrypt=True
# )

# Test the connection
# print("HANA DB Version:", hana_connection.hana_version())
# print("Current Schema:", hana_connection.get_current_schema())

hana_connection = dbapi.connect(
    address=hdb_host_address,
    port=hdb_port,
    user=hdb_user,
    password=hdb_password,
    #encrypt=True
    autocommit=True,
    sslValidateCertificate=False,
)





hdb_host_address: bfff8255-c34a-41cb-a822-bf9b5f56fb16.hna0.prod-eu20.hanacloud.ondemand.com
hdb_user: DBADMIN
hdb_port: 443


In [17]:
#A0.4 Setup LLM-Connection to SAP AI-HUB

import os
import dotenv
from gen_ai_hub.proxy.langchain.openai import ChatOpenAI
from gen_ai_hub.proxy.langchain.openai import OpenAI

# Lade aicore_model_name aus der Umgebungskonfiguration
aicore_model_name = str(os.getenv("AICORE_DEPLOYMENT_MODEL"))

# Überprüfe, ob die Variable definiert ist
if not aicore_model_name:
    raise ValueError(f"""Parameter LLM-Model-Name {aicore_model_name} fehlt in der Umgebungskonfiguration.""")

llm = ChatOpenAI(proxy_model_name=aicore_model_name)
#llm = OpenAI(proxy_model_name=aicore_model_name)

if not llm:
    raise ValueError(f"""Parameter LLM-Model-Name {aicore_model_name} fehlt in der Umgebungskonfiguration.""")
else:
    print(f"""Parameter LLM-Model-Name: {aicore_model_name} wurde erfolgreich geladen.""")


Parameter LLM-Model-Name: gpt-4 wurde erfolgreich geladen.


### processing functions Modul A

- function A2: load the pdf-file with accounting assignment guide data and splitt the data into text_chunk

- function A3: vectorize the splitted data with embedding function 

- function A4.1: create a LangChain VectorStore interface for the HANA database and specify the table

- function A4.2: delete existing documents from the table and load embeddings to SAP HANA-Tabele



In [8]:
# function A2: load the pdf-file containing the accounting assignment guide data from a folder and splitt the data into text_chunks
# the funktion uses the os modules to read the file and split the data with langChain-modules

from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import PyPDFLoader

def load_and_split_pdf(file_path):
    # Check if the file exists
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"The file {file_path} does not exist.")
    
    # Load the PDF file
    loader = PyPDFLoader(file_path)
    documents = loader.load()
    
    # Split the documents into text_chunks
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    text_chunks = text_splitter.split_documents(documents)
    
    return text_chunks

if __name__ == "__main__":
    
    # Test the function with a sample PDF file path
    file_path = "data/sample_accounting_guide.pdf"
    try:
        text_chunks = load_and_split_pdf(file_path)
        print(f"Number of text_chunks created: {len(text_chunks)}")
        print(f"First text_chunks: {text_chunks[0]}")
    except FileNotFoundError as e:
        print(e)


Number of text_chunks created: 70
First text_chunks: page_content='Kontierungshandbuch 
 
 
1 von 70 
Inhaltsverzeichnis 
1 Einleitung 3 
1.1 Inhalt und Aufbau des Handbuchs 3 
1.2 Kontennomenklatur 4 
1.3 Buchungslogik und Zusatzkontierungen 5 
1.4 Bedienungshinweise zum Kontierungshandbuch 6 
1.4.1 Hyperlinks 6 
1.4.2 Farbschema 6 
2 Bilanz: Aktiva 7 
2.1 Anlagevermögen 7 
2.1.1 Immaterielle Vermögensgegenstände 7 
2.1.1.1 Selbsterstellte Software 7 
2.1.2 Sachanlagen 9 
2.1.2.1 Außerplanmäßige Abschreibung von Sachanlagen 9 
2.1.2.2 Aktivierung der Verbindlichkeiten aus Capital Lease 10 
2.1.2.2.1 Aktivierung der Verbindlichkeiten aus Capital Lease 11 
2.1.2.2.2 Aktivierung der Verbindlichkeiten aus Capital Lease 11 
2.2 Umlaufvermögen 12 
2.2.1 Vorräte 12 
2.2.1.1 Ermittlung des beizulegenden Werts (Marktwert) 13 
2.2.1.2 Niederstwerttest (Lower of Cost or Market Prinzip) 13 
2.2.2 Forderungen und sonstige Vermögensgegenstände 14 
2.2.2.1 Forderungen gegen verbundene Unternehmen 15

In [ ]:
# function A3 - vectorize the splitted data with embedding function 
# Load the text file containing the accounting assignment guide data from a folder and splitt the data into chunks. 
# Use the embedding models from SAP AI-hub for embedding.

# Initialize embeddings

from gen_ai_hub.proxy.langchain.init_models import init_embedding_model

ai_core_embedding_model_name = str(os.getenv('AICORE_DEPLOYMENT_MODEL_EMBEDDING'))
 
try:
    embeddings = init_embedding_model(ai_core_embedding_model_name)
    print("Embedding model initialized successfully.")
except Exception as e:
    print("Embedding model not initialized.")
    print(e)


Embedding model initialized successfully.


In [10]:
# function A4.1 - Create a LangChain VectorStore interface for the HANA database and specify the table (collection) 
# to use for accessing the vector embeddings
# check table creation with sap-hana-database explorer: select * from ACCOUNTING_ASSIGN_SUPPORT_TABLE_DBADMIN

from langchain_community.vectorstores.hanavector import HanaDB

vector_table_name = str(os.getenv('hdb_table_name')) + "_" + str(os.getenv('hdb_user')) 

hana_database = HanaDB(
    embedding = embeddings, 
    connection = hana_connection, 
    table_name = vector_table_name
)

try:
    print(f"""
    Successfully created SAP HANA VectorStore interface: {hana_database.connection}
    and SAP HANA table: {vector_table_name}.
    """)
except Exception as e:
    print(e)



    Successfully created SAP HANA VectorStore interface: <dbapi.Connection Connection object : bfff8255-c34a-41cb-a822-bf9b5f56fb16.hna0.prod-eu20.hanacloud.ondemand.com,443,DBADMIN,DB@hmx04,True>
    and SAP HANA table: ACCOUNTING_ASSIGN_SUPPORT_TABLE_DBADMIN.
    


In [ ]:
# function A4.2 - delete existing documents from the table and load embeddings to SAP HANA-Table

# Delete already existing documents from the SAP HANA table
hana_database.delete(filter={})

# add the loaded document text_chunks
hana_database.add_documents(text_chunks)

print(f"Successfully added {len(text_chunks)} document chunks to the database.")
print("table-name: ",hana_database.table_name)
print("Successfully connected to the HANA Cloud database.")

Successfully added 70 document chunks to the database.
table-name:  ACCOUNTING_ASSIGN_SUPPORT_TABLE_DBADMIN
Successfully connected to the HANA Cloud database.


In [15]:
# check function A4.2 - query to the table to verify embeddings

cursor = hana_connection.cursor()
sql = f'SELECT VEC_TEXT, TO_NVARCHAR(VEC_VECTOR) FROM "{hana_database.table_name}"'

cursor.execute(sql)
vectors = cursor.fetchall()

print(vectors[:1])

# for vector in vectors:
#     print(vector)

[('Kontierungshandbuch \n \n \n1 von 70 \nInhaltsverzeichnis \n1 Einleitung 3 \n1.1 Inhalt und Aufbau des Handbuchs 3 \n1.2 Kontennomenklatur 4 \n1.3 Buchungslogik und Zusatzkontierungen 5 \n1.4 Bedienungshinweise zum Kontierungshandbuch 6 \n1.4.1 Hyperlinks 6 \n1.4.2 Farbschema 6 \n2 Bilanz: Aktiva 7 \n2.1 Anlagevermögen 7 \n2.1.1 Immaterielle Vermögensgegenstände 7 \n2.1.1.1 Selbsterstellte Software 7 \n2.1.2 Sachanlagen 9 \n2.1.2.1 Außerplanmäßige Abschreibung von Sachanlagen 9 \n2.1.2.2 Aktivierung der Verbindlichkeiten aus Capital Lease 10 \n2.1.2.2.1 Aktivierung der Verbindlichkeiten aus Capital Lease 11 \n2.1.2.2.2 Aktivierung der Verbindlichkeiten aus Capital Lease 11 \n2.2 Umlaufvermögen 12 \n2.2.1 Vorräte 12 \n2.2.1.1 Ermittlung des beizulegenden Werts (Marktwert) 13 \n2.2.1.2 Niederstwerttest (Lower of Cost or Market Prinzip) 13 \n2.2.2 Forderungen und sonstige Vermögensgegenstände 14 \n2.2.2.1 Forderungen gegen verbundene Unternehmen 15 \n2.2.2.2 Umbuchung von langfristigen

# Check retrievaleval for the embeddings in the SAP-Hana-Database

This code snippet integrates various components from the langchain library to create a retrieval-based question-answering (QA) system. Here's a breakdown of the key parts and their functionality:

Retriever Initialization: The db.as_retriever function is used to initialize a retriever object with specific search arguments ('k':20), which likely defines the number of search results to consider.

Prompt Template : The PromptTemplate was defined in the previous step that instructs how to use the context to answer a question. It emphasizes not to fabricate answers if the information is unavailable. The template also outlines the structure for the expected JSON output with various product and supplier details.The prompt template is crucial for guiding the model's responses, ensuring that the answers are relevant and accurate based on the retrieved information.

Once the retriever and prompt template are set up, the next step involves using the LLM (Language Model) to generate answers based on the retrieved documents. This process typically includes passing the retrieved context to the LLM along with the user's query, allowing it to formulate a coherent and contextually appropriate response.After that, the LLM processes the prompt and generates a response, which can then be formatted and returned to the user, ensuring a seamless interaction with the QA system.

In [ ]:
# function A4.3: Check retrievaleval for the embeddings in the SAP-Hana-Database

import os
from langchain import PromptTemplate

template = """Question: {question}

Answer: Let's think step by step."""

prompt = PromptTemplate(template=template, input_variables=["question"])
llm_chain = prompt | llm

question = "What NFL team won the Super Bowl in the year Justin Bieber was born?"

print(llm_chain.invoke({'question': question}))

content='First, Justin Bieber was born on March 1, 1994. So we need to check who won the Super Bowl in 1994. The Super Bowl in 1994 (Super Bowl XXVIII) was won by the Dallas Cowboys.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 51, 'prompt_tokens': 33, 'total_tokens': 84, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gpt-4', 'system_fingerprint': None, 'id': 'chatcmpl-BIuQgEz94b2gXSzGkzxf1X3KUWbeh', 'finish_reason': 'stop', 'logprobs': None} id='run-1933ad7d-1c4b-4bf4-9813-4ff8f40d6e7c-0' usage_metadata={'input_tokens': 33, 'output_tokens': 51, 'total_tokens': 84, 'input_token_details': {}, 'output_token_details': {}}


In [33]:
# check function 4.2 : check retrieval from SAP HANA DB with prompt using chain_type="map_reduce"

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

map_template = """
Analysiere den folgenden Kontext und extrahiere relevante Informationen zur Kontierung:

{context}

Frage: {question}

Gib die relevanten Informationen in einem kurzen Zwischenergebnis zurück.

"""

reduce_template = """
Basierend auf den folgenden Zwischenergebnissen, erstelle eine finale Antwort:

{summerization}

Frage: {question}

Formatiere die Ergebnisse in einer Liste von JSON-Elementen mit den folgenden Schlüsseln:
"Geschäftsfall"
"Konto Soll"
"Konto Haben"

Die Ergebnisse dürfen keine json markdown codeblock syntax enthalten.
Wenn keine relevanten Informationen gefunden wurden, gib an dass Du keine Antwort kennst.
"""

MAP_PROMPT = PromptTemplate(template=map_template, input_variables=["context", "question"])
REDUCE_PROMPT = PromptTemplate(template=reduce_template, input_variables=["summerization", "question"])

chain_type_kwargs = {
    "question_prompt": MAP_PROMPT,
    "reduce_prompt": REDUCE_PROMPT
}

question = "Finde Kontierung für die Umbuchung von langfristigen Forderungen"

retriever = hana_database.as_retriever(search_kwargs={'k':20})

question_answer_retriever = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type='map_reduce',
     verbose=True
)

answer = question_answer_retriever.run(question)
print(answer)



> Entering new RetrievalQA chain...

> Finished chain.
Die Kontierung für die Umbuchung von langfristigen Forderungen ist folgendermaßen:

Kurzfristiger Anteil an sonstigen Aktiva 
Kontonummer: 26690000 

an 

Langfristige Darlehensforderungen
Kontonummer: 26515000


In [31]:
# check function 4.2 : check retrieval from SAP HANA DB with prompt using chain_type="stuff"
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

prompt_template = """Verwende den folgenden Kontext, um die Frage am Ende zu beantworten. Wenn du die Antwort nicht kennst,
    sage einfach, dass du es nicht weißt. Versuche nicht, eine Antwort zu erfinden. Formatiere die Ergebnisse als Liste von JSON-Elementen mit den folgenden Schlüsseln:

    "Geschäftsfall",
    "Konto Soll",
    "Konto Haben"

    Füge keine JSON-Markdown-Codeblock-Syntax in die Ergebnisse ein.

    {context}

    Frage: {question}
"""

PROMPT = PromptTemplate(template=prompt_template, 
                       input_variables=["context", "question"]
                      )
    
chain_type_kwargs = {"prompt": PROMPT}

question = "Finde Kontierung für die Buchung von Rückstellungen"

count_retrieved_documents = 10

retriever = hana_database.as_retriever(search_kwargs={'k': count_retrieved_documents})
# hint: k smaller than 20 -> to much tokens

question_answer_retriever = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs=chain_type_kwargs,
    verbose=True
)

answer = question_answer_retriever.run(question)
print(answer)



> Entering new RetrievalQA chain...

> Finished chain.
[
{
"Geschäftsfall": "Bildung von Aufwandsrückstellungen",
"Konto Soll": "L160501: LC Instandhaltungskosten",
"Konto Haben": "L3909101: LC Sonstige Rückstellungen"
},
{
"Geschäftsfall": "Auflösung der HGB-Rückstellung bei Eintritt des Rückstellungsgrundes",
"Konto Soll": "61605010: Instandhaltungskosten",
"Konto Haben": "44001000: Kreditor"
},
{
"Geschäftsfall": "Zahlung von Rückstellungen",
"Konto Soll": "44001000: Kreditor",
"Konto Haben": "12000000: Bank"
},
{
"Geschäftsfall": "Aufzinsung von langfristigen Rückstellungen",
"Konto Soll": "75901000: Sonstige Zinsaufwendungen",
"Konto Haben": "39120200: Rückstellungen für Einzelgarantien"
},
{
"Geschäftsfall": "Abzinsung von langfristigen Rückstellungen",
"Konto Soll": "39120200: Rückstellungen für Einzelgarantien",
"Konto Haben": "57901000: Sonstige Zinserträge Inland"
},
{
"Geschäftsfall": "Auflösung der Aufwandsrückstellung",
"Konto Soll": "L3909101: LC Sonstige Rückstellungen

In [34]:
# check function 4.2 : check retrieval from SAP HANA DB with prompt using chain_type="map_reduce" (prompt-finetuning)

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

# Prompt für die Map-Phase
map_prompt_template = """Analysiere den folgenden Kontext und extrahiere relevante Kontierungsinformationen für den Geschäftsfall.
Wenn keine relevanten Informationen im Kontext gefunden werden können, gib die Antwort zurück: "Ich habe keine Kontierungsinformationen zum Geschäftsfall gefunden".

{context}

Frage: {question}
"""

# Prompt für die Combine/Reduce-Phase
combine_prompt_template = """
- Fasse die folgenden Kontierungsinformationen zusammen und entferne Duplikate.
- Gib nur die relevantesten und eindeutigsten Kontierungen zurück. 
- Wenn keine relevanten Informationen im Kontext gefunden werden können, gib die Antwort zurück: "Ich habe keine Kontierungsinformationen zum Geschäftsfall gefunden".
- Wenn relevante Informationen gefunden wurden gib diese Informationen im folgenden Struktur aus:
    ## Geschäftsfall: <Bezeichnung des Geschäftsfalls>
    ## Kontierung
    Konto-Soll: <Konto-Soll> - <Bezeichnung Konto-Soll> AN Konto-Haben: <Konto-Haben> - <Bezeichnung Konto-Haben>

Beispiel:

-----------------------------
## Geschäftsfall: Bildung von Rückstellungen
    ## Kontierung
    Konto-Soll: L160501 - LC Instandhaltungskosten (Gebäude) AN Konto-Haben: L3909101 - LC Sonstige Rückstellungen
-----------------------------

{summaries}

Frage: {question}
"""

# Korrekte Struktur für chain_type_kwargs
chain_type_kwargs = {
    "question_prompt": PromptTemplate(
        template=map_prompt_template,
        input_variables=["context", "question"]
    ),
    "combine_prompt": PromptTemplate(
        template=combine_prompt_template,
        input_variables=["summaries", "question"]
    )
}

question = "Finde Kontierung für die Buchung von Bildung von Rückstellungen"

retriever = hana_database.as_retriever(search_kwargs={'k': 20})

question_answer_retriever = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="map_reduce",
    retriever=retriever,
    chain_type_kwargs=chain_type_kwargs,
    verbose=True
)

answer = question_answer_retriever.run(question)
print(answer)



> Entering new RetrievalQA chain...

> Finished chain.
## Geschäftsfall: Bildung von Rückstellungen
    ## Kontierung
    Konto-Soll: L160501 - LC Instandhaltungskosten (Gebäude) AN Konto-Haben: L3909101 - LC Sonstige Rückstellungen
    Konto-Soll: 43000000 - GuV-Konto Instandhaltungskosten AN Konto-Haben: L160501 - LC Instandhaltungskosten (Gebäude)
    Konto-Soll: 91000000 - Eröffnungsbilanzkonto AN Konto-Haben: L160501 - LC Instandhaltungskosten (Gebäude)
    Konto-Soll: L3909101 - LC Sonstige Rückstellungen AN Konto-Haben: 90000000 - Schlussbilanzkonto

## Geschäftsfall: Buchung von Pensionsrückstellungen (US GAAP)
    ## Kontierung
    Konto-Soll: 64501010 - US Veränderung der Pensionsrückstellungen AN Konto-Haben: 37001010 - US Pensionsrückstellungen  

## Geschäftsfall: Buchung von Pensionsrückstellungen (HGB)
    ## Kontierung
    Konto-Soll: L6450101 - Veränderung der Pensionsrückstellungen AN Konto-Haben: L3700101 - Pensionsrückstellungen 

## Geschäftsfall: Aufwandsrückste